<hr style="border-width:3px; border-color:coral"/>

# Running shell commands from Python using `subprocess`

<hr style="border-width:3px; border-color:coral"/>

Best way to run command shell/terminal commands from Python is to use the 'subprocess' module.
You can read more about the 'subprocess' command [here](https://docs.python.org/3/library/subprocess.html).

We will use the `subprocess.run()` command.  Let's try it with a very simple command.  

    $ echo "Hello, World!"
    Hello, World!

We first need to import the 'subprocess' module. 

In [1]:
import subprocess

In the spirit of "RTFM is for sissy-weasels", let's try something obvious. 

In [2]:
subprocess.run('echo "Hello, World!"')

FileNotFoundError: [Errno 2] No such file or directory: 'echo "Hello, World!"': 'echo "Hello, World!"'

That went well.  Let's try reading up on the `subprocess` command....

Let's specify that this is a 'shell' command : 

In [3]:
subprocess.run('echo "Hello, World!"',shell=True)

CompletedProcess(args='echo "Hello, World!"', returncode=0)

Better,  but not what we expect.  Before parsing the output, let's modify how we supply the input command. 

It will be preferable to split the shell command into 'tokens', or a list of arguments.  Why? 

* Increases portability when handling different operating systems and shells, 
* Allows for easy replacement of individual arguments to get more flexible behavior.  

In [4]:
# Split command into an argument list.  Here, we use a Python list using []
cmd = 'echo "Hello, World!"'

# Split or 'tokenize' the command into a list of arguments
cmd_args = ['echo','"Hello, World!"'] 

# Use subprocess again.  Note that we don't need the 'shell=True' argument anymore.
subprocess.run(args=cmd_args)

CompletedProcess(args=['echo', '"Hello, World!"'], returncode=0)

Now let's work on the getting the output we expect.  What we need to do is to **capture** the output.  To do this, we add one more argument to our `subprocess.run()` command.  

And, because we are lazy, we use the `shlex` module to split our command line for us. 

In [5]:
import shlex

cmd = 'echo "Hello, World!"'
cmd_args = shlex.split(cmd)    # Split or 'tokenize' the command into list of arguments

# Add arguments to parse output
output = subprocess.run(args=cmd_args,capture_output=True)
output.stdout    # Returns a string with weird characters

b'Hello, World!\n'

To get rid of the byte encoding, we use one more command.  This, and the 
print command, we will get the results that we want.

In [6]:
cmd = 'echo "Hello, World!"'
cmd_args = shlex.split(cmd)

output = subprocess.run(args=cmd_args,capture_output=True,text=True)
print(output.stdout)    # Print adds an extra '\n' (in addition to one in output.stdout)

Hello, World!



It may bug you that there seems to be an extra blank line after the output (it bugs me).  

The string `output.stdout` already includes a line feed `'\n'` and the print statement adds a second newline character, and so we get that extra blank line in the output.

We can get rid of the extra newline that the print statement provides by supplying the `end` argument to the `print` statement

In [7]:
print(output.stdout,end='')   

Hello, World!


<hr style="border-width:3px; border-color:black"/>

<hr style="border-width:3px; border-color:coral"/>

## Complete examples using shell commands
<hr style="border-width:3px; border-color:coral"/>


Finally, here are some examples of other shell commands we might want to use.  

* Ex 1 : Get current working directory

      $ pwd
      /Users/calhoun/www/parallel-programming/python/parallel

* Ex 2 : Find files in your current directory : 

      $ ls -lh
      total 76K
      drwxr-xr-x 24 calhoun staff  768 Jan 15 07:40 Demos_Fall2019/
      -rw-r--r--  1 calhoun staff  67K Jan 15 09:17 multiprocessing_demo0.ipynb
      -rw-r--r--  1 calhoun staff 5.7K Jan 15 09:27 shellcommands_demo.ipynb
          
* Ex 3 : What shell are we using?  Query an environment variable

      $ echo $SHELL
      /opt/local/bin/bash

* Ex 4 : Time the sleep command

      $ time sleep 10           

      real  0m10.022s
      user  0m0.002s
      sys   0m0.004s

In [9]:
# Import necessary modules (already done above, but put here as a reminder)
import subprocess
import shlex

In [10]:
# Ex 1 : Get current working directory
cmd_args = shlex.split('pwd')
output = subprocess.run(args=cmd_args,capture_output=True,text=True)
print(output.stdout,end='')

/Users/calhoun/ME471-571-spring2020/Week_01/Wed


In [11]:
# Ex 2 : List files in current directory
cmd_args = shlex.split('ls -lh')
output = subprocess.run(args=cmd_args,capture_output=True,text=True)
print(output.stdout,end='')

total 328K
-rw-r--r-- 1 calhoun staff 149K Jan 15 14:46 mean_demo0.ipynb
-rw-r--r-- 1 calhoun staff  71K Jan 15 15:00 mean_demo1.ipynb
-rw-r--r-- 1 calhoun staff  62K Jan 15 13:50 multiprocessing_demo0.ipynb
-rw-r--r-- 1 calhoun staff 5.1K Jan 15 14:28 multiprocessing_demo1.ipynb
-rw-r--r-- 1 calhoun staff 1.4K Jan 15 14:59 multiprocessing_demo2.ipynb
-rw-r--r-- 1 calhoun staff 1.4K Jan 15 15:17 overview_01_15_2020.ipynb
-rw-r--r-- 1 calhoun staff  15K Jan 15 15:26 shell_demo0.ipynb
-rw-r--r-- 1 calhoun staff 6.6K Jan 15 14:58 shell_demo1.ipynb


In [12]:
# Ex 3 : Query environment variable
import os

cmd = ['echo',os.environ["SHELL"]] 
output = subprocess.run(args=cmd,capture_output=True,text=True)
print(output.stdout,end='')

/opt/local/bin/bash


In [13]:
# Ex 4 : Time to sleep.  Note use of 'stderr' rather than 'stdout'. 
cmd = shlex.split('time sleep 10')
output = subprocess.run(args=cmd,capture_output=True,text=True)
print(output.stderr,end='')

       10.01 real         0.00 user         0.00 sys


In [15]:
# Ex 4 (revisited) : Time to sleep.  Note the use of the 'shell=True' argument.  
cmd = shlex.split('time sleep 60')
output = subprocess.run(args=cmd,capture_output=True,text=True, shell=True)
print(output.stderr,end='')


real	0m0.001s
user	0m0.000s
sys	0m0.000s


Why aren't we getting the correct timing results here?  ..... Stay tuned!